# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f011c08cee0>
├── 'a' --> tensor([[ 0.5532,  0.8548,  0.1377],
│                   [-0.2362,  0.4633,  0.3557]])
└── 'x' --> <FastTreeValue 0x7f011c08c580>
    └── 'c' --> tensor([[-2.0259,  0.7207, -0.3728, -0.5239],
                        [-0.5684, -0.4932, -1.3866,  0.5380],
                        [ 0.4958, -0.5602,  0.4363,  1.1144]])

In [4]:
t.a

tensor([[ 0.5532,  0.8548,  0.1377],
        [-0.2362,  0.4633,  0.3557]])

In [5]:
%timeit t.a

63.1 ns ± 0.0471 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f011c08cee0>
├── 'a' --> tensor([[ 1.4386, -0.8349,  0.1385],
│                   [-0.1684, -0.7690, -0.2482]])
└── 'x' --> <FastTreeValue 0x7f011c08c580>
    └── 'c' --> tensor([[-2.0259,  0.7207, -0.3728, -0.5239],
                        [-0.5684, -0.4932, -1.3866,  0.5380],
                        [ 0.4958, -0.5602,  0.4363,  1.1144]])

In [7]:
%timeit t.a = new_value

79.9 ns ± 0.0608 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.5532,  0.8548,  0.1377],
               [-0.2362,  0.4633,  0.3557]]),
    x: Batch(
           c: tensor([[-2.0259,  0.7207, -0.3728, -0.5239],
                      [-0.5684, -0.4932, -1.3866,  0.5380],
                      [ 0.4958, -0.5602,  0.4363,  1.1144]]),
       ),
)

In [10]:
b.a

tensor([[ 0.5532,  0.8548,  0.1377],
        [-0.2362,  0.4633,  0.3557]])

In [11]:
%timeit b.a

59.7 ns ± 0.0978 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.2876, -0.5780, -0.8726],
               [ 0.0911,  1.3760, -1.1555]]),
    x: Batch(
           c: tensor([[-2.0259,  0.7207, -0.3728, -0.5239],
                      [-0.5684, -0.4932, -1.3866,  0.5380],
                      [ 0.4958, -0.5602,  0.4363,  1.1144]]),
       ),
)

In [13]:
%timeit b.a = new_value

518 ns ± 5.46 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

848 ns ± 1.31 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

10.7 µs ± 28.6 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

145 µs ± 627 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

145 µs ± 644 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f0071d06850>
├── 'a' --> tensor([[[ 0.5532,  0.8548,  0.1377],
│                    [-0.2362,  0.4633,  0.3557]],
│           
│                   [[ 0.5532,  0.8548,  0.1377],
│                    [-0.2362,  0.4633,  0.3557]],
│           
│                   [[ 0.5532,  0.8548,  0.1377],
│                    [-0.2362,  0.4633,  0.3557]],
│           
│                   [[ 0.5532,  0.8548,  0.1377],
│                    [-0.2362,  0.4633,  0.3557]],
│           
│                   [[ 0.5532,  0.8548,  0.1377],
│                    [-0.2362,  0.4633,  0.3557]],
│           
│                   [[ 0.5532,  0.8548,  0.1377],
│                    [-0.2362,  0.4633,  0.3557]],
│           
│                   [[ 0.5532,  0.8548,  0.1377],
│                    [-0.2362,  0.4633,  0.3557]],
│           
│                   [[ 0.5532,  0.8548,  0.1377],
│                    [-0.2362,  0.4633,  0.3557]]])
└── 'x' --> <FastTreeValue 0x7f010cfc7220>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32 µs ± 32.6 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f0071d06ac0>
├── 'a' --> tensor([[ 0.5532,  0.8548,  0.1377],
│                   [-0.2362,  0.4633,  0.3557],
│                   [ 0.5532,  0.8548,  0.1377],
│                   [-0.2362,  0.4633,  0.3557],
│                   [ 0.5532,  0.8548,  0.1377],
│                   [-0.2362,  0.4633,  0.3557],
│                   [ 0.5532,  0.8548,  0.1377],
│                   [-0.2362,  0.4633,  0.3557],
│                   [ 0.5532,  0.8548,  0.1377],
│                   [-0.2362,  0.4633,  0.3557],
│                   [ 0.5532,  0.8548,  0.1377],
│                   [-0.2362,  0.4633,  0.3557],
│                   [ 0.5532,  0.8548,  0.1377],
│                   [-0.2362,  0.4633,  0.3557],
│                   [ 0.5532,  0.8548,  0.1377],
│                   [-0.2362,  0.4633,  0.3557]])
└── 'x' --> <FastTreeValue 0x7f011c077e50>
    └── 'c' --> tensor([[-2.0259,  0.7207, -0.3728, -0.5239],
                        [-0.5684, -0.4932, -1.3866,  0.5380],
                 

In [23]:
%timeit t_cat(trees)

29.9 µs ± 65.9 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

56.3 µs ± 90.3 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[-2.0259,  0.7207, -0.3728, -0.5239],
                       [-0.5684, -0.4932, -1.3866,  0.5380],
                       [ 0.4958, -0.5602,  0.4363,  1.1144]],
              
                      [[-2.0259,  0.7207, -0.3728, -0.5239],
                       [-0.5684, -0.4932, -1.3866,  0.5380],
                       [ 0.4958, -0.5602,  0.4363,  1.1144]],
              
                      [[-2.0259,  0.7207, -0.3728, -0.5239],
                       [-0.5684, -0.4932, -1.3866,  0.5380],
                       [ 0.4958, -0.5602,  0.4363,  1.1144]],
              
                      [[-2.0259,  0.7207, -0.3728, -0.5239],
                       [-0.5684, -0.4932, -1.3866,  0.5380],
                       [ 0.4958, -0.5602,  0.4363,  1.1144]],
              
                      [[-2.0259,  0.7207, -0.3728, -0.5239],
                       [-0.5684, -0.4932, -1.3866,  0.5380],
                       [ 0.4958, -0.5602,  0.4363,  1.1144]],

In [26]:
%timeit Batch.stack(batches)

77.4 µs ± 86.5 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[-2.0259,  0.7207, -0.3728, -0.5239],
                      [-0.5684, -0.4932, -1.3866,  0.5380],
                      [ 0.4958, -0.5602,  0.4363,  1.1144],
                      [-2.0259,  0.7207, -0.3728, -0.5239],
                      [-0.5684, -0.4932, -1.3866,  0.5380],
                      [ 0.4958, -0.5602,  0.4363,  1.1144],
                      [-2.0259,  0.7207, -0.3728, -0.5239],
                      [-0.5684, -0.4932, -1.3866,  0.5380],
                      [ 0.4958, -0.5602,  0.4363,  1.1144],
                      [-2.0259,  0.7207, -0.3728, -0.5239],
                      [-0.5684, -0.4932, -1.3866,  0.5380],
                      [ 0.4958, -0.5602,  0.4363,  1.1144],
                      [-2.0259,  0.7207, -0.3728, -0.5239],
                      [-0.5684, -0.4932, -1.3866,  0.5380],
                      [ 0.4958, -0.5602,  0.4363,  1.1144],
                      [-2.0259,  0.7207, -0.3728, -0.5239],
                   

In [28]:
%timeit Batch.cat(batches)

143 µs ± 219 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

324 µs ± 4.3 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
